In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
import re
import math
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import csv

In [2]:
def fill_input(nombre_comercial, camara, actividad, entidad):
    t=3 #tiempo de espera
    #Rellenando el Nombre Comercial
    elem_nombre = driver.find_element_by_xpath('//*[@id="entradaNombreComercial"]')
    elem_nombre.clear()
    elem_nombre.click()
    time.sleep(t)
    ActionChains(driver).send_keys(nombre_comercial).send_keys(Keys.RETURN).perform()
    #Llenando la camara
    driver.find_element_by_xpath('//*[@id="vs1__combobox"]').click()
    time.sleep(t)
    ActionChains(driver).send_keys(camara).send_keys(Keys.RETURN).perform()
    #Llenando la actividad
    driver.find_element_by_xpath('//*[@id="vs2__combobox"]').click()
    time.sleep(t)
    ActionChains(driver).send_keys(actividad).send_keys(Keys.RETURN).perform()
    #Rellenando la Entidad 
    driver.find_element_by_xpath('//*[@id="vs3__combobox"]').click()
    time.sleep(t)
    ActionChains(driver).send_keys(entidad).send_keys(Keys.RETURN).perform()    
    #Clic en el botón buscar
    driver.find_element_by_id('botonGuardar').click()
    return

def info_extract():
    #Entrando a la página con info de la empresa
    driver.find_element_by_xpath('//*[@id="__BVID__72"]/tbody/tr/td[4]/div/div/div/button').click()
    time.sleep(4)
    #Abriendo la pestaña los datos generales
    i=0
    for i in range(3):
        driver.find_element_by_css_selector('.btn.btn-primary.active.center-block.btn-secondary.btn-block.collapsed').click()
        time.sleep(2)
    datos_brutos = driver.find_elements_by_css_selector('div.card-body')
    datos_limpios = [element.text.split('\n') for element in datos_brutos]
    return datos_limpios

#    Otra forma de hacerlo pero el numero de columnas no coinicide en todos los casos
#    page_source = driver.page_source
#    soup = BeautifulSoup(page_source, 'html')
#    datos_generales = soup.select('p')
#    datos_generales_clean = [element.text for element in datos_generales]
#    tabla_datos= pd.DataFrame(datos_generales_clean, index=columns).T
    


In [3]:
#Llamando al webdriver Chrome y abriendo la página del siem
driver = webdriver.Chrome('/Users/lupita/Documents/IronHack/complementos/chromedriver')

In [ ]:
driver.get('https://siem.economia.gob.mx/ui/pubconsultaestablecimientos')

In [ ]:
#Creando encabezado de tabla 
columns=['Nombre Comercial','Tipo de vialidad','Nombre de la vialidad','Número exterior',\
        'Edificio. Piso o Nivel','Número Interior','Número de Local','Codigo Postal',\
        'Tipo de asentamiento humano','Nombre asentamiento','Centro Comercial, Parque Industrial, otro',\
        'Nombre','Entrevialidades Tipo1','Nombre1','Tipo 2','Nombre 2','Vialidad Posterior Tipo',\
        'Nombre','Descripción Ubicación Territorial','Entidad Federativa','Delegación o Municipio',\
        'Localidad', 'Datos de Contacto. Telefóno', 'Extensión','E-mail','Otro','Sitio Web',\
        'Perfil General de la Empresa. Tipo', 'Tipo de Instalaciones','¿Exporta?',\
        'Paises','¿Importa?','Paises','Actividad principal', 'Producto o Servicio principal',\
        'Porcentaje','Producto o Servicio principal(2)','Porcentaje','Producto o Servicio principal(2)','Porcentaje',\
        'Actividad(Código)','Principales productos o servicois que ofrece','Destino',\
        'Principales insumos que demanda','Origen','Insumos (2)','Origen (2)',\
        'Insumos (3)','Origen (3)','Facebook', 'Twitter','Grupo Empresarial','Cámara y/o Asociación',\
        'Clúster','Cadena Productiva', 'Distintivo','Nombre Distintivo','Drop']

hoteles_otros_servicios = pd.DataFrame( columns = columns) 
hoteles_otros_servicios

In [4]:
#Importando la lista de empresas
lista_empresas = pd.read_csv ('lista_empresas0.csv')
lista_empresas.drop(['Unnamed: 0'],axis=1)
lista_empresas.columns
len(lista_empresas)

649

In [7]:
driver.get('https://siem.economia.gob.mx/ui/pubconsultaestablecimientos')
time.sleep(5)
entidad = 'Ciudad de México'
datos =[]
registros_multiples = []
registros_no_encontrados = []
for i in range(60,100,1):
    #Defininedo las entradas del formulario
    nombre_comercial = lista_empresas['Nombre Comercial'][i]
    camara = lista_empresas['Cámara'][i]
    actividad = lista_empresas['Actividad'][i]
    #Llenando el formulario
    fill_input(nombre_comercial, camara, actividad, entidad)
    #obteniendo el número de registros encontrados en esa busqueda.
    time.sleep(10)
    registros = driver.find_elements_by_css_selector('h3.text-secondary')
    leyenda_registros = [element.text for element in registros]
    numero =int(re.search('\d+',leyenda_registros[0]).group())
    if numero == 0:
        print("No se encontró registro")
        registros_no_encontrados.append(nombre_comercial)
    elif numero == 1:
        info = info_extract()
        datos.append(info)
         
        #hoteles_otros_servicios = pd.concat([hoteles_otros_servicios, info])
        driver.get('https://siem.economia.gob.mx/ui/pubconsultaestablecimientos')
        time.sleep(8)
    else :
        print("Hay mas de un registro")
        registros_multiples.append(nombre_comercial)
datos_df = pd.DataFrame(datos)
datos_df.to_csv('Datos_SIEM.csv', encoding='utf-8-sig')
registros_no_encontrados =pd.DataFrame(registros_no_encontrados)
registros_no_encontrados.to_csv('Registros_no_encontrados.csv', encoding='utf-8-sig')
registros_multiples =pd.DataFrame(registros_multiples)
registros_multiples.to_csv('registros_multiples.csv', encoding='utf-8-sig')



Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro
Hay mas de un registro


AttributeError: 'NoneType' object has no attribute 'group'

In [10]:
datos_df = pd.DataFrame(datos)
datos_df.to_csv('Datos_SIEM8.csv', encoding='utf-8-sig')


In [9]:
registros_no_encontrados =pd.DataFrame(registros_no_encontrados)
registros_no_encontrados.to_csv('Registros_no_encontrados.csv', encoding='utf-8-sig')
registros_multiples =pd.DataFrame(registros_multiples)
registros_multiples.to_csv('registros_multiples.csv', encoding='utf-8-sig')

#Abriendo la pestaña los datos generales


In [ ]:
time.sleep(0)
    #Abriendo la pestaña de perfil general
driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[4]/div/button').click()
time.sleep(0)
    #Abrieno la pestaña de actividad económica
driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[5]/div/button').click()
time.sleep(0)
datos_brutos = driver.find_elements_by_css_selector('div.card-body')
datos_limpios = [element.text.split('\n') for element in datos_brutos]

//*[@id="app"]/div[2]/div/div[3]/div/button

In [ ]:
#time.sleep(1)
    #Entrando a la página con info de la empresa
#    driver.find_element_by_xpath('//*[@id="__BVID__72"]/tbody/tr/td[4]/div/div/div/button').click()
 #   time.sleep(2)
    #Abriendo la pestaña los datos generales
driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[2]/div/button').click()
time.sleep(2)
    #Abriendo la pestaña de perfil general
driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[3]/div/button').click()
time.sleep(2)
    #Abriendo la pestaña de actividad económica
driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[4]/div/button').click()
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html')
datos_generales = soup.select('p')
datos_generales_clean = [element.text for element in datos_generales]
tabla_datos= pd.DataFrame(datos_generales, index=columns).T

In [ ]:
hoteles_otros_servicios = pd.concat([hoteles_otros_servicios, info])
print(hoteles_otros_servicios)

In [ ]:
driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[2]/div/button').click()
#//*[@id="app"]/div[2]/div/div[2]/div/button